In [ ]:
#Date: Dec 11, 2024
#Author: Sonal Allana
#Purpose: To generate SHAP explanations from the models

In [1]:
import numpy as np
from numpy import argmax
from numpy import loadtxt
from numpy import savetxt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import shap
import os
import miattack

2024-10-06 17:23:25.726151: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
#Options (1) baseline_nn (2) dp_nn (3) syn_nn
base_folder = "dp_nn"

if base_folder == "dp_nn":
    #Options (1) nm4000 (2) nm500 (3) nm66 (4) nm15
    nm_folder = "nm4000"   #set the correct noise multiplier
elif base_folder == "syn_nn":
    #Options (1) ctgan (2) gausscopula (3) tvae 
    syndataType = "tvae"

In [3]:
#Options (1) adult (2) credit (3) compas (4) hospital
dataset_name = "hospital"

In [4]:
#for shap explanations
expl = "shap"
folder = "shap"

In [5]:
for i in range(1,6):
    #Load model and train-test files
    if base_folder == "dp_nn": #account for noise multiplier folder
        basepath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,nm_folder,i)
        fmodel = "model_dp_iter{0}.keras".format(i)
    elif base_folder == "syn_nn": #account for syn type folder
        basepath = "../models/{0}/{1}/{2}/iter{3}/".format(base_folder,dataset_name,syndataType,i)
        fmodel = "model_wodp_iter{0}.keras".format(i)
    else:
        basepath = "../models/{0}/{1}/iter{2}/".format(base_folder,dataset_name, i)
        fmodel = "model_wodp_iter{0}.keras".format(i)
    model = tf.keras.models.load_model(basepath + fmodel,compile=False)
    model.summary()
    X_train = loadtxt(basepath + 'X_train.csv',delimiter=',')
    X_test = loadtxt(basepath + 'X_test.csv',delimiter=',')
    Y_train = loadtxt(basepath + 'Y_train.csv',delimiter=',')
    Y_test = loadtxt(basepath + 'Y_test.csv',delimiter=',')
    Z_train = loadtxt(basepath + 'Z_train.csv',delimiter=',')
    Z_test = loadtxt(basepath + 'Z_test.csv',delimiter=',')
    
    #50% of the test set is used for training and remaining 50% for testing the attack model
    (X_adv_train, X_adv_test, Y_adv_train, Y_adv_test, Z_adv_train, Z_adv_test)  = train_test_split(X_test, Y_test, Z_test, test_size=0.5, random_state=1337)

    input_train = np.array(X_adv_train)
    input_test = np.array(X_adv_test)
    background_data = shap.kmeans(input_train,5) #5 is the number of clusters
    explainer = shap.KernelExplainer(model,data=background_data)
    attributions_train = explainer.shap_values(input_train)
    print(np.shape(attributions_train))
    
    attributions_test = explainer.shap_values(input_test)
    print(np.shape(attributions_test))
    
    if not os.path.exists(basepath + folder):
        os.mkdir(basepath + folder)
    
    savetxt(basepath + folder + '/attributions_train.csv',attributions_train[0],delimiter=',')
    savetxt(basepath + folder + '/attributions_test.csv',attributions_test[0],delimiter=',')
    savetxt(basepath + folder + '/X_adv_train.csv',X_adv_train,delimiter=',')
    savetxt(basepath + folder + '/X_adv_test.csv',X_adv_test,delimiter=',')
    savetxt(basepath + folder + '/Y_adv_train.csv',Y_adv_train,delimiter=',')
    savetxt(basepath + folder + '/Y_adv_test.csv',Y_adv_test,delimiter=',')
    savetxt(basepath + folder + '/Z_adv_train.csv',Z_adv_train,delimiter=',')
    savetxt(basepath + folder + '/Z_adv_test.csv',Z_adv_test,delimiter=',')
    
    print("Iteration ",i,":")
    #Attack the first sensitive attribute
    print("Model inversion on sensitive attribute 1: ")
    Z_adv_train_s1 = Z_adv_train[:,0]
    Z_adv_test_s1 = Z_adv_test[:,0]
    modinv_obj1 = miattack.miattack_explanations(attributions_train[0],attributions_test[0],Z_adv_train_s1,Z_adv_test_s1)
    modinv_obj1.printMetrics()
    
    #Attack the second sensitive attribute
    print("\nModel inversion on sensitive attribute 2: ")
    Z_adv_train_s2 = Z_adv_train[:,1]
    Z_adv_test_s2 = Z_adv_test[:,1]
    modinv_obj2 = miattack.miattack_explanations(attributions_train[0],attributions_test[0],Z_adv_train_s2,Z_adv_test_s2)
    modinv_obj2.printMetrics()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 120)]             0         
                                                                 
 dense1 (Dense)              (None, 1024)              123904    
                                                                 
 dense2 (Dense)              (None, 512)               524800    
                                                                 
 dense3 (Dense)              (None, 256)               131328    
                                                                 
 dense4 (Dense)              (None, 100)               25700     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 805833 (3.07 MB)
Trainable params: 805833 (3.07

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md



  0%|          | 0/4925 [00:00<?, ?it/s]

(1, 4925, 120)


  0%|          | 0/4925 [00:00<?, ?it/s]

...

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


  0%|          | 0/4925 [00:00<?, ?it/s]

(1, 4925, 120)


  0%|          | 0/4925 [00:00<?, ?it/s]

(1, 4925, 120)
Iteration  5 :
Model inversion on sensitive attribute 1: 
Precision: 0.9431
Recall: 0.9248
F1-Score: 0.9339
FPR: 0.0473
Attacker's advantage: 0.8775
Attack success: 93.99%

Model inversion on sensitive attribute 2: 
Precision: 0.9997
Recall: 0.9963
F1-Score: 0.9980
FPR: 0.0009
Attacker's advantage: 0.9954
Attack success: 99.70%
